# JPEG Compression with Zonal Coding Quantization
stough 202-
DIP 8.9

In this activity we're going to synthesize a bit of the [JPEG](https://en.wikipedia.org/wiki/JPEG) encoding standard, specifically block transform encoding and zonal coding quantization. We will look to understand how the quantization affects the image reconstruction.

Read up in DIP 8.9 or [elsewhere](https://en.wikipedia.org/wiki/JPEG#JPEG_codec_example) if needed.

## Imports
We're going to take advantage of both [`view_as_blocks`](https://scikit-image.org/docs/dev/api/skimage.util.html?highlight=view_as_blocks#skimage.util.view_as_blocks) and the [Discrete Cosine transform](https://en.wikipedia.org/wiki/Discrete_cosine_transform) for this.

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair,
                       vis_surface)

from wavelet_utils import (make_haar_matrix,
                           make_random_basis,
                           make_klt_basis,
                           make_dct_matrix,
                           make_standard_matrix,
                           vis_blocks)

from skimage.util import view_as_blocks
from skimage.util import montage
from skimage.transform import resize, rescale
from ipywidgets import VBox, HBox, FloatSlider

## JPEG Compression
Basically
- Take the image in $[0,255]$ form and subtract 128. This is to center around 0. Make sure you change the `dtype` when you do this, or else you won't be able to represent negatives.
- Transform each 8x8 block using the DCT transform matrix (see `make_dct_matrix`)
- Quantize the transform coefficients according the [Q matrix](https://en.wikipedia.org/wiki/JPEG#Quantization). This is, divide the block coefficients and store the result in integer form.
- These quantized transform blocks represent how the compressed image would be stored or transmitted. We won't go further than just having this representation, as the point of the exercise is to understand how this quantization affects the reconstruction. 
- Reconstruction: for each transform block, remultiply according to the Q matrix and then invert the transform process, to reconstruct the as-though-compressed image block.

We'll define the Q matrix for ease of use.

In [2]:
H = make_dct_matrix(8)
vis_blocks(H)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
B = 215 * np.ones((8,8))
B

array([[215., 215., 215., 215., 215., 215., 215., 215.],
       [215., 215., 215., 215., 215., 215., 215., 215.],
       [215., 215., 215., 215., 215., 215., 215., 215.],
       [215., 215., 215., 215., 215., 215., 215., 215.],
       [215., 215., 215., 215., 215., 215., 215., 215.],
       [215., 215., 215., 215., 215., 215., 215., 215.],
       [215., 215., 215., 215., 215., 215., 215., 215.],
       [215., 215., 215., 215., 215., 215., 215., 215.]])

In [4]:
B128 = B.astype('float')-128
B128

array([[87., 87., 87., 87., 87., 87., 87., 87.],
       [87., 87., 87., 87., 87., 87., 87., 87.],
       [87., 87., 87., 87., 87., 87., 87., 87.],
       [87., 87., 87., 87., 87., 87., 87., 87.],
       [87., 87., 87., 87., 87., 87., 87., 87.],
       [87., 87., 87., 87., 87., 87., 87., 87.],
       [87., 87., 87., 87., 87., 87., 87., 87.],
       [87., 87., 87., 87., 87., 87., 87., 87.]])

In [5]:
T = np.matmul(H, np.matmul(B128,H.transpose()))
T

array([[ 6.96000000e+02,  6.78279942e-14, -5.58952915e-14,
         3.79962375e-14,  3.45420341e-14,  1.93749409e-13,
        -1.34949447e-13, -9.02018118e-14],
       [ 3.14853305e-14,  5.52202634e-30, -5.71924156e-30,
         3.64848169e-30,  1.38050658e-30,  2.49477261e-29,
        -8.75142567e-30, -5.54667824e-30],
       [-6.93491172e-14, -5.71924156e-30,  4.28943117e-30,
        -2.93357649e-30, -4.38803879e-30, -2.38383905e-29,
         1.75090143e-29,  1.17527949e-29],
       [ 3.52598442e-14,  5.22620350e-30, -3.72243740e-30,
         1.46678825e-30,  2.19401939e-30,  1.34969171e-29,
        -9.93779851e-30, -4.69310609e-30],
       [ 2.43132171e-14,  2.16936749e-30, -2.81031697e-30,
         1.79958894e-30,  1.13398755e-30,  8.50490663e-30,
        -9.90390215e-30, -2.64391663e-30],
       [ 1.89021637e-13,  1.94256998e-29, -1.59497814e-29,
         1.07359039e-29,  9.29376754e-30,  5.74759125e-29,
        -3.99638167e-29, -2.57458315e-29],
       [-1.38739241e-13, -1.328737

In [17]:
Tq = np.round(T/Q)
Tq

array([[44.,  0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0., -0.],
       [-0., -0.,  0., -0., -0., -0.,  0.,  0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0., -0.],
       [ 0.,  0., -0.,  0.,  0.,  0., -0., -0.],
       [-0., -0.,  0., -0., -0., -0.,  0.,  0.],
       [-0., -0.,  0., -0., -0., -0.,  0.,  0.]])

## Exercise: Interactive Quantization Demo
Put together an interactive demo that shows the original image, its current reconstruction, and a view of the currently quantized transform coefficients. This demo should then interact with a `FloatSlider` that affects the multiple applied to the Q quantization matrix. This demo would show how you could increase the compression (increasing the multiple) and what that increased compression would do to the image reconstruction.

In [7]:
Q = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
              [12, 12, 14, 19, 26, 58, 60, 55],
              [14, 13, 16, 24, 40, 57, 69, 56],
              [14, 17, 22, 29, 51, 87, 80, 62],
              [18, 22, 37, 56, 68, 109, 103, 77],
              [24, 35, 55, 64, 81, 104, 113, 92],
              [49, 64, 78, 87, 103, 121, 120, 101],
              [72, 92, 95, 98, 112, 100, 103, 99]])

In [8]:
H2 = make_dct_matrix(8)

In [9]:
I = plt.imread("../dip_pics/mountainSpring.jpg")
I = ((I - I.min()) / (I.max() - I.min()) * 255).astype('uint8') # Normalization
vis_hists(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
arr_info(I)

((1200, 1600, 3), dtype('uint8'), 0, 255)

In [11]:
block_shape = (8,8,3)
Blocks = view_as_blocks(I, block_shape) # list of original images
blocklist = np.reshape(Blocks, [Blocks.shape[0]*Blocks.shape[1]] + list(block_shape))
#arr_info(block_view)

In [12]:
def make_quantization(mult, Blocks, D, Q):
    global blocklist
    
    ReconsQ = blocklist.copy().astype('float')
    TransQ = blocklist.copy().astype('float')
    
    for index, block in enumerate(blocklist):
        ReconsQ_block = block.copy().astype('float')
        temp = block.astype('float')-128
        for channel in range(3):
            TransQ[index][...,channel] = np.round(np.matmul(D, np.matmul(temp[...,channel],D.transpose()))/(mult*Q)) # Generate the coefficient matrix for a channel, then quantize it by a multiple m
            ReconsQ_block[...,channel] = np.matmul(D.transpose(), np.matmul(TransQ[index][...,channel]*mult*Q, D)) + 128 # Reconstruct by reverse process
        
        ReconsQ[index] = ReconsQ_block # Update the reconstructed Image
        

    return np.clip(montage(ReconsQ, grid_shape = Blocks.shape[:2], multichannel=True, padding_width = 0, fill=[1,1,1]),0,255)/255, np.clip(montage(TransQ, grid_shape = Blocks.shape[:2], multichannel=True, padding_width = 0, fill=[1,1,1]),0,255)/255



### Interactive Demo

In [13]:
plt.ioff()
plt.clf()

coeff_slider = FloatSlider(
    orientation = 'horizontal',
    value = 1.00,
    min = 0.20,
    max = 5.00,
    step = 0.01,
    description = 'Multiple of Q'
)

fig_args = {'num':' ', 'frameon':True, 'sharex':True, 'sharey':True}
fig, ax = plt.subplots(1,3, figsize=(8,3), **fig_args) 

Original = ax[0].imshow(I[440:640,720:880,:])
ax[0].set_title("Original Image")
Reconstruct = ax[1].imshow(I[440:640,720:880,:])
ReconTitle = ax[1].set_title("Reconstructed using quantized DCT")
Coeff = ax[2].imshow(I[440:640,720:880,:])
text = ax[2].set_title("Quantized Coefficients")



def update(change):
    global Original, Reconstruct, Coeff, text
    ReconQ, TransQ = make_quantization(change.new, Blocks,H2,Q)
    Reconstruct.set_array(ReconQ[440:640,720:880,:])
    ReconTitle.set_text(f'Quantized Reconstruction \n with multiple: { format(change.new)}')
    Coeff.set_array(TransQ[440:640,720:880,:])
    text.set_text(f'Coeff with multiple \n of { format(change.new)}')
    fig.canvas.draw()
    fig.canvas.flush_events()

coeff_slider.observe(update, names = 'value')

plt.tight_layout()
VBox([coeff_slider, fig.canvas])
